pca_glove_250features 

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Glove/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Glove/glove_withstopwordchangelevels_250_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,240,241,242,243,244,245,246,247,248,249
0,0,-4.966919,-5.388837,-1.189118,-0.407037,-2.644020,-1.298720,0.659784,0.431904,2.106731,...,0.114988,0.123762,-0.032639,0.070436,0.049549,-0.089165,-0.138055,0.013068,-0.061136,0.092182
1,1,-1.553555,-1.746878,-4.481486,3.524568,-3.414967,5.271519,-8.076717,-1.099770,0.737332,...,0.033921,0.016709,-0.037102,-0.003686,0.005487,-0.015048,-0.013855,0.031764,-0.023644,-0.044472
2,2,-4.461761,-0.418745,1.780855,-3.794923,3.386196,1.567599,0.000003,1.121250,0.174587,...,0.020874,0.049946,-0.052654,-0.043281,0.056923,0.076338,0.039088,0.046623,-0.085975,-0.038907
3,3,20.157560,-4.956659,5.808807,3.058060,0.412849,-3.123765,6.337488,-1.362786,12.199962,...,-0.013915,-0.042262,0.079377,0.006127,0.017598,0.074609,-0.020520,-0.013550,-0.060148,-0.024650
4,4,-4.792284,-0.317271,3.858270,-4.363634,0.199257,3.157568,-0.218778,2.016098,0.123783,...,0.142763,-0.066568,0.117039,-0.064026,-0.034239,0.104690,0.068404,-0.065884,0.018160,-0.167891


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,-4.966919,-5.388837,-1.189118,-0.407037,-2.644020,-1.298720,0.659784,0.431904,2.106731,-0.900328,...,0.114988,0.123762,-0.032639,0.070436,0.049549,-0.089165,-0.138055,0.013068,-0.061136,0.092182
1,-1.553555,-1.746878,-4.481486,3.524568,-3.414967,5.271519,-8.076717,-1.099770,0.737332,-1.403242,...,0.033921,0.016709,-0.037102,-0.003686,0.005487,-0.015048,-0.013855,0.031764,-0.023644,-0.044472
2,-4.461761,-0.418745,1.780855,-3.794923,3.386196,1.567599,0.000003,1.121250,0.174587,1.283436,...,0.020874,0.049946,-0.052654,-0.043281,0.056923,0.076338,0.039088,0.046623,-0.085975,-0.038907
3,20.157560,-4.956659,5.808807,3.058060,0.412849,-3.123765,6.337488,-1.362786,12.199962,-5.851593,...,-0.013915,-0.042262,0.079377,0.006127,0.017598,0.074609,-0.020520,-0.013550,-0.060148,-0.024650
4,-4.792284,-0.317271,3.858270,-4.363634,0.199257,3.157568,-0.218778,2.016098,0.123783,-1.045854,...,0.142763,-0.066568,0.117039,-0.064026,-0.034239,0.104690,0.068404,-0.065884,0.018160,-0.167891


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 250), (524, 250))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.34625173197488274


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.512295,0.512295,0.564432,0.500786,0.318702,0.318702,0.344793,0.310273


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9832    0.9751    0.9791       361
           2     0.9834    0.9638    0.9735       553
           3     0.9437    0.9869    0.9649       306

    accuracy                         0.9730      1220
   macro avg     0.9701    0.9753    0.9725      1220
weighted avg     0.9734    0.9730    0.9730      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3750    0.1146    0.1756       157
           2     0.4435    0.8439    0.5814       237
           3     0.2800    0.0538    0.0903       130

    accuracy                         0.4294       524
   macro avg     0.3662    0.3375    0.2824       524
weighted avg     0.3824    0.4294    0.3380       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.512295,0.512295,0.564432,0.500786,0.318702,0.318702,0.344793,0.310273
1,RandomForest,0.972951,0.972951,0.973405,0.973007,0.429389,0.429389,0.382395,0.337984


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6105    0.4515    0.5191       361
           2     0.5572    0.7577    0.6421       553
           3     0.6269    0.4118    0.4970       306

    accuracy                         0.5803      1220
   macro avg     0.5982    0.5403    0.5528      1220
weighted avg     0.5904    0.5803    0.5693      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2736    0.1847    0.2205       157
           2     0.4390    0.6076    0.5097       237
           3     0.2444    0.1692    0.2000       130

    accuracy                         0.3721       524
   macro avg     0.3190    0.3205    0.3101       524
weighted avg     0.3412    0.3721    0.3462       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.527869,0.527869,0.565215,0.472622,0.414122,0.414122,0.325075,0.324854
1,RandomForest,0.973770,0.973770,0.973806,0.973764,0.421756,0.421756,0.359941,0.345894
2,Adaboost,0.580328,0.580328,0.590433,0.569344,0.372137,0.372137,0.341182,0.346242


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9799    0.9446    0.9619       361
           2     0.9365    0.9873    0.9613       553
           3     0.9758    0.9216    0.9479       306

    accuracy                         0.9582      1220
   macro avg     0.9641    0.9512    0.9570      1220
weighted avg     0.9592    0.9582    0.9581      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3069    0.1975    0.2403       157
           2     0.4639    0.7046    0.5595       237
           3     0.2222    0.1077    0.1451       130

    accuracy                         0.4046       524
   macro avg     0.3310    0.3366    0.3150       524
weighted avg     0.3569    0.4046    0.3610       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.527869,0.527869,0.565215,0.472622,0.414122,0.414122,0.325075,0.324854
1,RandomForest,0.973770,0.973770,0.973806,0.973764,0.421756,0.421756,0.359941,0.345894
2,Adaboost,0.580328,0.580328,0.590433,0.569344,0.372137,0.372137,0.341182,0.346242
3,GBM,0.958197,0.958197,0.959205,0.958107,0.404580,0.404580,0.356906,0.361034


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6184    0.7452    0.6759       361
           2     0.7870    0.6148    0.6904       553
           3     0.6034    0.6961    0.6464       306

    accuracy                         0.6738      1220
   macro avg     0.6696    0.6854    0.6709      1220
weighted avg     0.6911    0.6738    0.6751      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3109    0.3822    0.3429       157
           2     0.4869    0.3924    0.4346       237
           3     0.2143    0.2308    0.2222       130

    accuracy                         0.3492       524
   macro avg     0.3374    0.3351    0.3332       524
weighted avg     0.3665    0.3492    0.3544       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.512295,0.512295,0.564432,0.500786,0.318702,0.318702,0.344793,0.310273
1,RandomForest,0.972951,0.972951,0.973405,0.973007,0.429389,0.429389,0.382395,0.337984
2,SVM,0.673770,0.673770,0.691074,0.675056,0.349237,0.349237,0.366533,0.354414


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4857    0.4709    0.4782       361
           2     0.5795    0.7251    0.6442       553
           3     0.5281    0.3072    0.3884       306

    accuracy                         0.5451      1220
   macro avg     0.5311    0.5011    0.5036      1220
weighted avg     0.5388    0.5451    0.5309      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3333    0.2994    0.3154       157
           2     0.4448    0.5781    0.5028       237
           3     0.2000    0.1154    0.1463       130

    accuracy                         0.3798       524
   macro avg     0.3260    0.3309    0.3215       524
weighted avg     0.3507    0.3798    0.3582       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.527869,0.527869,0.565215,0.472622,0.414122,0.414122,0.325075,0.324854
1,RandomForest,0.973770,0.973770,0.973806,0.973764,0.421756,0.421756,0.359941,0.345894
2,Adaboost,0.580328,0.580328,0.590433,0.569344,0.372137,0.372137,0.341182,0.346242
3,GBM,0.958197,0.958197,0.959205,0.958107,0.404580,0.404580,0.356906,0.361034
4,SVM,0.697541,0.697541,0.699016,0.692806,0.366412,0.366412,0.353655,0.357976
5,KNN,0.545082,0.545082,0.538845,0.530917,0.379771,0.379771,0.350672,0.358206


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4662    0.3629    0.4081       361
           2     0.5241    0.7071    0.6020       553
           3     0.5596    0.3529    0.4329       306

    accuracy                         0.5164      1220
   macro avg     0.5166    0.4743    0.4810      1220
weighted avg     0.5159    0.5164    0.5022      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2426    0.2102    0.2253       157
           2     0.4675    0.6076    0.5284       237
           3     0.2875    0.1769    0.2190       130

    accuracy                         0.3817       524
   macro avg     0.3326    0.3316    0.3242       524
weighted avg     0.3555    0.3817    0.3608       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.527869,0.527869,0.565215,0.472622,0.414122,0.414122,0.325075,0.324854
1,RandomForest,0.973770,0.973770,0.973806,0.973764,0.421756,0.421756,0.359941,0.345894
2,Adaboost,0.580328,0.580328,0.590433,0.569344,0.372137,0.372137,0.341182,0.346242
3,GBM,0.958197,0.958197,0.959205,0.958107,0.404580,0.404580,0.356906,0.361034
4,SVM,0.697541,0.697541,0.699016,0.692806,0.366412,0.366412,0.353655,0.357976
5,KNN,0.545082,0.545082,0.538845,0.530917,0.379771,0.379771,0.350672,0.358206
6,GNB,0.516393,0.516393,0.515878,0.502203,0.381679,0.381679,0.355488,0.360843
